In [2]:
import yfinance as yf
import pandas as pd

In [3]:
df = yf.download(
    tickers="AAPL",
    interval="1m",
    period='8d'
)

/var/folders/r6/jv1hv35d4rg8crmf3xb99ykc0000gn/T/ipykernel_7033/200115773.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed


In [4]:
df = df.droplevel("Ticker", axis=1)
df

Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-09-10 13:30:00+00:00,231.240005,232.339996,230.740005,232.024994,3415708
2025-09-10 13:31:00+00:00,231.009995,231.589996,230.610001,231.289993,521090
2025-09-10 13:32:00+00:00,230.679993,231.399994,230.550003,231.029999,492306
2025-09-10 13:33:00+00:00,230.595001,230.979996,230.399994,230.699997,548846
2025-09-10 13:34:00+00:00,230.645004,230.820007,230.529999,230.580002,275368
...,...,...,...,...,...
2025-09-19 15:11:00+00:00,242.710007,242.820007,242.611603,242.750000,81850
2025-09-19 15:12:00+00:00,242.660004,242.830002,242.613602,242.699997,81610
2025-09-19 15:13:00+00:00,242.399994,242.690002,242.399994,242.660004,125343


In [13]:
from backtesting import Strategy, Backtest
import datetime as dt

In [ ]:
class OpeningRangeBreakout(Strategy):
    open_range_minutes = 5
    last_minute_bar_in_opening_range = dt.time(13, 30 + open_range_minutes)
    exit_minute_bar = dt.time(19, 59)
    
    def init(self):
        self.current_day        = None
        self.current_day_open   = None
        self.opening_range_high = None
        self.opening_range_low  = None
        
    def _reset_range(self, day, open):
        self.current_day        = day
        self.current_day_open   = open
        self.opening_range_high = None
        self.opening_range_low  = None
        
    def next(self):
        t = self.data.index[-1]
        current_bar_date = t.date()

        
        if self.current_day != current_bar_date:
            self._reset_range(current_bar_date, self.data.Open[-1])
            print("Resetting range for day:", current_bar_date)
            print(t.time())
            
        if t.time() <= self.last_minute_bar_in_opening_range:
            if self.opening_range_high is None:
                self.opening_range_high = self.data.High[-1]
                self.opening_range_low  = self.data.Low[-1]
            else:
                self.opening_range_high = max(self.opening_range_high, self.data.High[-1])
                self.opening_range_low  = min(self.opening_range_low,  self.data.Low[-1])
                
            print(f"Updated opening range: {self.opening_range_low} - {self.opening_range_high}")
            
        if t.time() == self.last_minute_bar_in_opening_range:
          print(f"opening range high is {self.opening_range_high}")
          print(f"opening range low is {self.opening_range_low}")

          if not self.position:
            if self.data.Close[-1] > self.current_day_open:
              print("going long")
              order = self.buy(size=250)
            elif self.data.Close[-1] < self.current_day_open:
              print("going short")
              order = self.sell(size=250)
            else:
              print("doji, doing nothing")

        if self.position and t.time() == self.exit_minute_bar:
          print("closing out position")
          self.position.close()
            
bt = Backtest(df, OpeningRangeBreakout, cash=25000)

stats = bt.run()

from bokeh.io import output_notebook
output_notebook()
bt.plot()
print(stats)

Resetting range for day: 2025-09-10
13:31:00
Updated opening range: 230.61000061035156 - 231.58999633789062
Updated opening range: 230.5500030517578 - 231.58999633789062
Updated opening range: 230.39999389648438 - 231.58999633789062
Updated opening range: 230.39999389648438 - 231.58999633789062
Updated opening range: 230.2100067138672 - 231.58999633789062
opening range high is 231.58999633789062
opening range low is 230.2100067138672
going short
Resetting range for day: 2025-09-11
13:30:00
Updated opening range: 226.64999389648438 - 227.77000427246094
Updated opening range: 226.64999389648438 - 227.77000427246094
Updated opening range: 226.64999389648438 - 227.77000427246094
Updated opening range: 226.64999389648438 - 227.77000427246094
Updated opening range: 226.64999389648438 - 227.77000427246094
Updated opening range: 226.64999389648438 - 227.77000427246094
opening range high is 227.77000427246094
opening range low is 226.64999389648438
going long
Resetting range for day: 2025-09-12

Loading BokehJS ...

Start                     2025-09-10 13:30...
End                       2025-09-19 15:15...
Duration                      9 days 01:45:00
Exposure Time [%]                         0.0
Equity Final [$]                      25000.0
Equity Peak [$]                       25000.0
Return [%]                                0.0
Buy & Hold Return [%]                 4.70291
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                      